# Introduction

This is Python notebook demonstrating how to integrate Gmail access with OpenAI capabilities to create a AI Agent that can read and write emails.

You should complete the following steps to get started:
- Enable API by following the [Enable the API - Quick start](https://developers.google.com/gmail/api/quickstart/python#enable_the_api).
- Set up your credentials explained in the [Gmail API docs](https://developers.google.com/gmail/api/quickstart/python#authorize_credentials_for_a_desktop_application).

1. Install required Python packages

In [4]:
!pip install --upgrade --quiet google-api-python-client google-auth-oauthlib google-auth-httplib2 langchain openai

2. Install load_dotenv and use it to load *OPENAI_API_KEY* from `.env` file

In [1]:
from dotenv import load_dotenv

if not load_dotenv("../config/.env"):
    raise Exception("Failed to load the .env file")

In [4]:
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="../config/token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="../config/cre.json",
)
api_resource = build_resource_service(credentials=credentials)

from langchain_community.agent_toolkits import GmailToolkit
toolkit = GmailToolkit(api_resource=api_resource)

3. List the Gmail tools supported by LangChain

In [29]:
tools = toolkit.get_tools()
tools

In [8]:
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType

4. Create a LangChain agent with OpenAI and any model you prefer

In [16]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

llm = ChatOpenAI(temperature=0)

agent = create_openai_functions_agent(llm, toolkit.get_tools(), prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    # This is set to False to prevent information about my email showing up on the screen
    # Normally, it is helpful to have it set to True however.
    verbose=True,
)

5. Ask the agent to work on your Gmail tasks

In [17]:
agent_executor.invoke(
    {"input": "Please send an email to 1193012432@qq.com with title 'Greetings from czy' and message 'This is czy, and nice to meet you'"}
)

In [10]:
agent.run(
    "Create a gmail draft for me to edit of a letter for a potential customer"
    " who has visited to my website and expressed interest in my product."
    " Under no circumstances may you send the message, however."
)

In [11]:
agent.run("Could you search in my drafts for the latest email?")

In [14]:
agent.run("Please send an email to zhengyang.cai@outlook.com with title 'Greetings from myself' and message 'This is czy, and nice to meet you'.")